In [1]:
#note: you may need to update your scikit-learn: pip install --upgrade sckikit-learn
import numpy as np
from numpy import genfromtxt
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
import csv
from jurrefuncs import *

#Retrieve the data from several trainingdata files and testdata files and put them in
#features, testfeatures and labels

dataFile = []
dataFile2 = []
rangenr = 10 #there are 9 data files, this nr is simply to go over all of them
for i in range(1, rangenr):
    dataFile.append(genfromtxt(r"feature extraction\dataset\clustering\sum up of the feature\sumupCluster_"+ str(i) +"_train_org.csv", delimiter=','))
for i in range(1, 9):    
    dataFile2.append(genfromtxt(r"feature extraction\dataset\clustering\concatence of the feature\cluster_"+ str(i) +"_train_org.csv", delimiter=','))
dataFile3 = genfromtxt(r"feature extraction\dataset\blocklization features\train_blocklization.csv", delimiter=',')

featurelength = len(dataFile[0]) -1

tempMatrix = np.array([dataFile[i][0:-1] for i in range(0, len(dataFile)) ])
tempMatrix2 = np.array([dataFile2[i][0:-1] for i in range(0, len(dataFile2)) ])
features1 = tempMatrix.reshape(-1, tempMatrix.shape[-1])
#features2 = tempMatrix2.reshape(-1, tempMatrix2.shape[-1])
featurestt2 = []
for i in range(0, len(tempMatrix2)):
    for j in tempMatrix2[i]:
        featurestt2.append( j )
        
features2 = featurestt2
features3 = dataFile3[0:-1]
features = np.concatenate((features1, features3[6:-1]) ,0) 

labels = list(dataFile[0][6])

#Also import the testdata

testdataFile = []
testdataFile2 = []
for i in range(1, rangenr):
    testdataFile.append(genfromtxt(r"feature extraction\dataset\clustering\sum up of the feature\sumupCluster_"+ str(i) +"_test_org.csv", delimiter=','))
for i in range(1, 9):
    testdataFile2.append(genfromtxt(r"feature extraction\dataset\clustering\concatence of the feature\cluster_"+ str(i) +"_test_org.csv",delimiter=','))
testdataFile3 = genfromtxt(r"feature extraction\dataset\blocklization features\test_blocklization.csv", delimiter=',')

testtempMatrix = np.array([testdataFile[i] for i in range(0, rangenr-1) ]) #does nothing
testtempMatrix2 = np.array([testdataFile2[i] for i in range(0, 8) ])
testfeatures1 = list(testtempMatrix.reshape(-1, testtempMatrix.shape[-1]))      
testfeatures2 = []
for i in range(0, len(testtempMatrix2)):
    for j in testtempMatrix2[i]:
        testfeatures2.append( j )
testfeatures3 = testdataFile3        
                
testfeaturestt = testfeatures1 
testfeatures = np.concatenate((testfeaturestt, testfeatures3[6:-1]) ,0) 

In [2]:
len(testfeatures)

4151

In [ ]:
x_names = ['mean', 'variance' , 'standard variance', 'median', 'skewness', 'kurtosis']
y_name = 'Brain age'
scatterPlotGrid(features_after_chi[6:12], labels, 3, 3, x_names, y_name); #second dataset starts at 54 ends at 270

In [5]:
#We have a crazy amount of features and many of them do not correlate to the age at all. 
#We want to do some basic filtering with a lasso algorithm
#Lasso will give us basic feature selection, but will also give us our first predictive model. 
#Lets generate this predictive model to get an idea how well it performs

#Try selecting features with lasso.
from sklearn.linear_model import Lasso
featurerange = len(features)
X_lasso_train = [[features[i][j]  for i in range(0, featurerange )] for j in range(0, len(features[0]))   ] 
X_lasso_test = [[testfeatures[i][j]  for i in range(0, featurerange )] for j in range(0, len(testfeatures[0]))   ] 
y_lasso = labels
alpha = [1e-1]#[1e-2]

lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e7)
lasfit = lassoreg.fit(X_lasso_train, y_lasso)
lassoselect = []
counttrue = 0

for i in lasfit.coef_:
    if i > 1e-5:
        lassoselect.append(True)
        counttrue +=1
    else:
        lassoselect.append(False)

print("score of lasso fit: " + str( lasfit.score(X_lasso_train, y_lasso) ) + " -- amount of features selected: " + str( counttrue) ) #testing on the training data doesnt mean that much

#Let's test our predictive model in kaggle, dont take the result too seriously!
lassosub = lasfit.predict(X_lasso_test)
c = csv.writer(open("submissionlassotemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(lassosub)):
    c.writerow([i+1 , lassosub[i]])
    

#lasso performs quite well on kaggle, about 75 points. However, this score may not be too reliable. 
#We will now make a list with all the features we keep after lasso
features_after_lasso_train = []
features_after_lasso_test = []
for i in range(0, len(lassoselect)):
    if lassoselect[i] == True:
        features_after_lasso_train.append(features[i])
        features_after_lasso_test.append(testfeatures[i])


 

score of lasso fit: 0.919910076469 -- amount of features selected: 11


In [ ]:
#Lasso is good, but it seems to make gaussian kernel fitting very hard. let us try another method
#first delete with low variance
#perhaps normalization is needed
from sklearn.feature_selection import *
X = [[features[i][j]  for i in range(0, len(features) )] for j in range(0, len(features[0])) ] 
sel = VarianceThreshold(threshold=(.8 * (1 - .8))) #.8 * (1 - .8)
X_lv = sel.fit_transform(X)
varselectlist = sel.get_support()
testfeatures_after_variance = []
for i in range(0, len(varselectlist)):
    if varselectlist[i]==True:
        testfeatures_after_variance.append(features[i])

In [ ]:
len(testfeatures_after_variance)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

X = [X_lv[i][:] +2e8 for i in range(0, len(X_lv))]
y = labels

chimod = SelectKBest(chi2, k=50)
X_new = chimod.fit_transform(X, y)

X_chi = [X_new[i][:] -2e8 for i in range(0, len(X_lv))]

chitruelist = chimod.get_support()

testfeatures_after_chi = []
for i in range(0, len(chitruelist)):
    if chitruelist[i]==True:
        testfeatures_after_chi.append(testfeatures_after_variance[i])
        
        
features_after_chi = [[X_chi[i][j] for i in range(0, len(X_chi) )] for j in range(0, len( X_chi[0] )) ]
    

In [ ]:
len(features_after_chi[0])

In [ ]:
X_lv[0][212]

In [ ]:
#Lasso did a good job. Lets try to fit the features that were selected by lasso with a gaussian kernel
#gaussian kernel
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold

featurerange = len(features_after_lasso_train)
fitdatarange = len(features_after_lasso_train[0])
#X_lasso_gaussian_train = [[features_after_lasso_train[i][j]  for i in range(0, featurerange )] for j in range(0, fitdatarange) ]
# X_chi 
#X_lasso_gaussian_test = [[features_after_lasso_test[i][j] for i in range(0, len(features_after_lasso_test))] for j in range(0, len(features_after_lasso_test[0])) ]
#[[features_after_lasso_test[i][j] for i in range(0, featurerange)] for j in range(0, len(features_after_lasso_test[0]))   ]
X_lasso_gaussian_train = [[features[i][j]  for i in range(0, 30 )] for j in range(0, len(features[0]))   ] 
X_lasso_gaussian_test = [[testfeatures[i][j]  for i in range(0, 30 )] for j in range(0, len(testfeatures[0]))   ] 
y = labels
#High C and low gamma seems to do it, but the algorithm will get stuck
parspace = np.logspace(-30, 30, num=30) #70 #this determines i or C 
parspace2 = np.logspace(-30, 30, num=30)#-5, -2 # this determines j or gamma
#minperf = 10000000000000

answerlist = []
ansnr = 0

kf = KFold(len(y), n_folds=3) #, n_folds=2
y_test = []
y_train = []
X_test = []
X_train = []

for train_index, test_index in kf:
    X_train_temp = []
    y_train_temp = []
    for i in train_index:
        X_train_temp.append(X_lasso_gaussian_train[i])
        y_train_temp.append(y[i])
    X_train.append(X_train_temp)
    y_train.append(y_train_temp)
    
    X_test_temp = []
    y_test_temp = []
    for i in test_index:
        X_test_temp.append(X_lasso_gaussian_train[i])
        y_test_temp.append(y[i])
    X_test.append(X_test_temp)
    y_test.append(y_test_temp) 



best = [-10,0,0,0,0]

countmonitor = 0
upd = [False, False, False]

for i in parspace:
    countmonitor += 1
    if np.ceil( len(parspace)/ countmonitor) == 2 and upd[1] == False:
        print("halfway")
        upd[1] = True
        
    if np.ceil( len(parspace)/ countmonitor) == 4 and upd[0] == False:
        print("quarter")
        upd[0] = True
        
    if np.ceil(4* len(parspace)/ countmonitor) == 3 and upd[2] == False:
        print("3/4")       
        upd[2] = True
        
    countjmon = 0
    for j in parspace2:
        countjmon += 1
        #print(countmonitor, countjmon)
        C = i
        gamma = j
        estimator = SVR(kernel='rbf', C=i, gamma=j, tol = 1e2)
        answerlist = []
        perflist = []
        for k in range(0, len(y_test)):
            #print(k)
            selector = estimator.fit(X_train[k], y_train[k])
            
            ans = selector.predict(X_lasso_gaussian_test)
            answerlist.append(ans)
            perf = selector.score(X_test[k], y_test[k])
            #print(perf)
            perflist.append(perf)
        
        perftot = np.mean(perflist)
        anstot = []
        for m in range(0, len(answerlist[0] )):
            sum1 = 0
            for n in range(0, len(answerlist)) :
                sum1 += answerlist[n][m]
            anstot.append(sum1/len(answerlist))    
                
        if perftot > best[0]:
            print(perftot)
            best = [perftot, anstot, perf, C, gamma] 
        
print("median performance = " + str(best[0]))        
print("second predicted value = " + str(best[1][0])) 
print("one selected performance = " + str(best[2])) 
print("with C = " + str(best[3])) 
print("with gamma = " + str(best[4])) 
        
ans2 = best[1] #answerlist[0] #selector.predict(sgtestData)
c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans2)):
    c.writerow([i+1 , ans2[i]]) 

In [ ]:
len(X_lasso_gaussian_train)

In [ ]:
#Gridsearch is also implemented by 
from __future__ import print_function

from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.svm import SVC

print(__doc__)

# Loading the Digits dataset
digits = datasets.load_digits()

###########################################################################################
parspaceC = np.logspace(-8, 8, num=30) #70 #this determines i or C 
parspacegamma = np.logspace(-8, 8, num=30)#-5, -2 # this determines j or gamma
######################################################################################
# To apply an classifier on this data, we need to flatten the image, to
# turn the data in a (samples, feature) matrix:
n_samples = len(digits.images)
X = X_lasso_gaussian_train
y = labels

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.5, random_state=0)

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': parspacegamma,
                     'C': parspaceC}]

scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_macro' % score)
    clf.fit(X, y)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Note the problem is too easy: the hyperparameter plateau is too flat and the
# output model is the same for precision and recall with ties in quality.

In [ ]:
y_pred

In [ ]:
#Recursive feature selection with cross validation, takes a very long time to evaluate
#When you are testing the data, use featurelength = low number
# in the final answer, use all features: featurerange = len(features)
featurerange = 30
fitdatarange = len(features[0])
Xstand = [[features[i][j]  for i in range(0, featurerange )] for j in range(0, fitdatarange)   ] 
Xextra = [[extrafeatures[i][j]  for i in range(0, featurerange )] for j in range(0, fitdatarange)   ] 
X = np.concatenate((Xstand, Xextra),1)
y = labels 
#when we use verbose, I think we get featurerange*(cv+1)/step outsputs of LibSVM
estimator = SVR(kernel="linear", verbose = True)  #Perhaps we should try a gaussian kernel 

    
#step is the amount of features to remove in one step, cv has to do with the nr of cross validations
selector = RFECV(estimator, step=1, cv=4) #lower cv if evaluation takes to long
selector = selector.fit(Xstand, y)
selectedBool = selector.support_ 

#Here the model (selector) is used to make prediction for the testdata. 
#The predictions are then written to a file
testDatastand = [[testfeatures[i][j] for i in range(0, featurerange)] for j in range(0, len(testfeatures[0]))   ]
testDataextra = [[extratestfeatures[i][j] for i in range(0, featurerange)] for j in range(0, len(testfeatures[0]))   ]
testData = np.concatenate((testDatastand, testDataextra),1)
ans = selector.predict(testDatastand)

c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans)):
    c.writerow([i+1 , ans[i]]) 
    
 
    
testlist = selector.predict(Xstand[150:276]) - labels[150:276]
testlist2 = testlist**2
thesum = 0
for i in testlist2:
    thesum += i
    
print(thesum)
#print('finish')

In [ ]:
selectedBool = selector.support_ 
selectedBool   


In [ ]:
featurerange = len(features)
testDatastand = [[testfeatures[i][j] for i in range(0, featurerange)] for j in range(0, len(testfeatures[0]))   ]
ans = lasfit.predict(testDatastand)

c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans)):
    c.writerow([i+1 , ans[i]]) 

In [ ]:
sgFeatures = []

for i in range(0, len(selectedBool)):
    if selectedBool[i]==True:
        sgFeatures.append(features[i])
        
        
sgTestfeatures = []

for i in range(0, len(selectedBool)):
    if selectedBool[i]==True:
        sgTestfeatures.append(testfeatures[i])  
        
        
lassoFeatures = []        
for i in range(0, len(lassoselect)):
    if lassoselect[i]==True:
        lassoFeatures.append(features[i])
        
        
lassoTestfeatures = []

for i in range(0, len(lassoselect)):
    if lassoselect[i]==True:
        lassoTestfeatures.append(testfeatures[i])     


In [ ]:
#testfield
len(sgX[0:150])

In [ ]:
#gaussian kernel
from sklearn.cross_validation import train_test_split
sgfeaturerange = len(sgFeatures)
sgfitdatarange = len(sgFeatures[0])
sgX = [[sgFeatures[i][j]  for i in range(0, sgfeaturerange )] for j in range(0, sgfitdatarange)   ]
sgXtrain = sgX[0:150]

y = labels
ytrain = y[0:150]
#when we use verbose, I think we get featurerange*(cv+1)/step outsputs of LibSVM

parspace = np.logspace(-1, 3, num=70) #70
parspace2 = np.logspace(-5, -2, num=70)#-5, -2
sgtestData = [[sgTestfeatures[i][j] for i in range(0, sgfeaturerange)] for j in range(0, len(sgTestfeatures[0]))   ]
minperf = 10000000000000

answerlist = []
ansnr = 0


sgX_train, sgX_test, y_train, y_test = train_test_split(sgX, y, test_size=0.2, random_state=True)
best = [0,0]


for i in parspace:
    for j in parspace2:
        estimator = SVR(kernel='rbf', C=i, gamma=j)  #Perhaps we should try a gaussian kernel 
        #step is the amount of features to remove in one step, cv has to do with the nr of cross validations
        selector = estimator.fit(sgX_train, y_train)
        #Here the model (selector) is used to make prediction for the testdata. 
        #The predictions are then written to a file
        

        ans = selector.predict(sgtestData)
         
        '''testlist = selector.predict(sgX_test) - y_test
        testlist2 = testlist**2
        thesum = 0
        for i in testlist2:
            thesum += i'''
    
        #print('the sum is ',thesum)
        #print(i, j)
        '''if (ans[1]-ans[40])**2 >=0 and selector.score(sgX_test, y_test)>= 0.9:#thesum<= 5700:
            print(ans[0:6], ans[30])
            print('ansnr = ', ansnr)
            ansnr +=1
            print('i en j', i, j)
            print(thesum)
            print(selector.score(sgX_test, y_test))
            minperfprev = minperf
            minperf = np.min([minperfprev,thesum])
            answerlist.append(ans)'''
        answerlist.append(ans)
        perf = selector.score(sgX_test, y_test)
        if perf > best[0]:
            best = [perf, ans] 
        
        
print(best[0])       
        
estimator = SVR(kernel='rbf', C=parspace[0], gamma=parspace2[0])  #Perhaps we should try a gaussian kernel 
#step is the amount of features to remove in one step, cv has to do with the nr of cross validations
selector = estimator.fit(sgX_test, y_test)
#Here the model (selector) is used to make prediction for the testdata. 
#The predictions are then written to a file
        
ans2 = best[1] #answerlist[0] #selector.predict(sgtestData)
c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans2)):
    c.writerow([i+1 , ans2[i]]) 



In [ ]:
#gaussian kernel with better cross validation
from sklearn.cross_validation import train_test_split
sgfeaturerange = len(sgFeatures)
sgfitdatarange = len(sgFeatures[0])
sgX = [[sgFeatures[i][j]  for i in range(0, sgfeaturerange )] for j in range(0, sgfitdatarange)   ]
sgXtrain = sgX[0:150]

y = labels
ytrain = y[0:150]
#when we use verbose, I think we get featurerange*(cv+1)/step outsputs of LibSVM

parspace = np.logspace(-1, 3, num=70)
parspace2 = np.logspace(-5, -2, num=70)#-5, -2
sgtestData = [[sgTestfeatures[i][j] for i in range(0, sgfeaturerange)] for j in range(0, len(sgTestfeatures[0]))   ]
minperf = 10000000000000

answerlist = []
ansnr = 0


from sklearn.cross_validation import KFold
kf = KFold(len(y), n_folds=5) #, n_folds=2
y_test = []
y_train = []
sgX_test = []
sgX_train = []

for train_index, test_index in kf:
    sgX_train_temp = []
    y_train_temp = []
    for i in train_index:
        sgX_train_temp.append(sgX[i])
        y_train_temp.append(y[i])
    sgX_train.append(sgX_train_temp)
    y_train.append(y_train_temp)
    
    sgX_test_temp = []
    y_test_temp = []
    for i in test_index:
        sgX_test_temp.append(sgX[i])
        y_test_temp.append(y[i])
    sgX_test.append(sgX_test_temp)
    y_test.append(y_test_temp) 



best = [0,0]


for i in parspace:
    for j in parspace2:
        estimator = SVR(kernel='rbf', C=i, gamma=j)
        answerlist = []
        perflist = []
        for k in range(0, len(y_test)):
            
            selector = estimator.fit(sgX_train[k], y_train[k])
            #Here the model (selector) is used to make prediction for the testdata. 
            #The predictions are then written to a file
            ans = selector.predict(sgtestData)
            answerlist.append(ans)
            perf = selector.score(sgX_test[k], y_test[k])
            perflist.append(perf)
        
        perftot = np.mean(perflist)
        anstot = []
        for i in range(0, len(answerlist[0] )):
            sum1 = 0
            for j in range(0, len(answerlist)) :
                sum1 += answerlist[j][i]
            anstot.append(sum1/len(answerlist))    
                
        if perftot > best[0]:
            best = [perftot, anstot, perf] 
        
print(best[0], best[1][0] ,best[2])       

        
ans2 = best[1] #answerlist[0] #selector.predict(sgtestData)
c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans2)):
    c.writerow([i+1 , ans2[i]]) 


In [ ]:
estimator = SVR(kernel='rbf', C=13.694452462581552, gamma=0.00033245979322709383)  
selector = estimator.fit(sgX, y)
ans = selector.predict(sgtestData)
ans[0:6]
ans3 = ans
c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans3)):
    c.writerow([i+1 , ans3[i]]) 



In [ ]:
from sklearn.cross_validation import KFold
kf = KFold(len(y), n_folds=5) #, n_folds=2
y_test = []
y_train = []
sgX_test = []
sgX_train = []

for train_index, test_index in kf:
    sgX_train_temp = []
    y_train_temp = []
    for i in train_index:
        sgX_train_temp.append(sgX[i])
        y_train_temp.append(y[i])
    sgX_train.append(sgX_train_temp)
    y_train.append(y_train_temp)
    
    sgX_test_temp = []
    y_test_temp = []
    for i in test_index:
        sgX_test_temp.append(sgX[i])
        y_test_temp.append(y[i])
    sgX_test.append(sgX_test_temp)
    y_test.append(y_test_temp) 

In [ ]:
#Try selecting features with lasso.
from sklearn.linear_model import Lasso
featurerange = 30
Xstand = [[features[i][j]  for i in range(0, featurerange )] for j in range(0, fitdatarange)   ] 
y = labels
alpha = [1e-3]#[1e-6]

lassoreg = Lasso(alpha=alpha,normalize=True, max_iter=1e7)
lasfit = lassoreg.fit(Xstand, y)
lassoselect = []
for i in lasfit.coef_:
    if i > 1e-5:
        lassoselect.append(True)
    else:
        lassoselect.append(False)

testlist = lasfit.predict(Xstand[150:276]) - labels[150:276]
testlist2 = testlist**2
thesum = 0
for i in testlist2:
    thesum += i
    
print(thesum)
print(lassoselect)
#print('finish')        
        

In [ ]:
caifa = 0 

In [ ]:
caifa

In [ ]:
caifa = 10

In [29]:
#The gaussian kernel doesnt seem to converge. Let us use our old algortithm with more elaborate information
from sklearn.cross_validation import train_test_split
sgfeaturerange = len(features_after_lasso_train)
sgfitdatarange = len(features_after_lasso_train[0])
sgX = [[features_after_lasso_train[i][j]  for i in range(0, sgfeaturerange )] for j in range(0, sgfitdatarange) ]
sgtestData = [[features_after_lasso_test[i][j] for i in range(0, len(features_after_lasso_test))] for j in range(0, len(features_after_lasso_test[0])) ]

y = labels


#X_lasso_gaussian_test = [[features_after_lasso_test[i][j] for i in range(0, len(features_after_lasso_test))] for j in range(0, len(features_after_lasso_test[0])) ]
#[[features_after_lasso_test[i][j] for i in range(0, featurerange)] for j in range(0, len(features_after_lasso_test[0]))   ]
#when we use verbose, I think we get featurerange*(cv+1)/step outsputs of LibSVM

parspace = np.logspace(3.3, 4.8, num=40) #70
parspace2 = np.logspace(-10, -1, num=40)#-5, -2



minperf = 10000000000000

answerlist = []
ansnr = 0


sgX_train, sgX_test, y_train, y_test = train_test_split(sgX, y, test_size=0.2, random_state=True)
best = [0,0]


for i in parspace:
    print('.')
    for j in parspace2:
        estimator = SVR(kernel='rbf', C=i, gamma=j)  #Perhaps we should try a gaussian kernel 
        #step is the amount of features to remove in one step, cv has to do with the nr of cross validations
        selector = estimator.fit(sgX_train, y_train)
        #Here the model (selector) is used to make prediction for the testdata. 
        #The predictions are then written to a file
        

        ans = selector.predict(sgtestData)
         
        testlist = selector.predict(sgX_test) - y_test
        testlist2 = testlist**2
        thesum = 0
        for k in testlist2:
            thesum += k
    
        #print('the sum is ',thesum)
        #print(i, j)
        if (ans[1]-ans[40])**2 >=0 and selector.score(sgX_test, y_test)>= 0.7:#thesum<= 5700:
            print(ans[0:6], ans[30])
            print('ansnr = ', ansnr)
            ansnr +=1
            print('i en j', i, j)
            print(thesum)
            print(selector.score(sgX_test, y_test))
            print('--------------------------------------')
            minperfprev = minperf
            minperf = np.min([minperfprev,thesum])
            answerlist.append(ans)
        answerlist.append(ans)
        perf = selector.score(sgX_test, y_test)
        if perf > best[0]:
            best = [perf, ans] 
        
        
print(best[0])       
        
estimator = SVR(kernel='rbf', C=parspace[0], gamma=parspace2[0])  #Perhaps we should try a gaussian kernel 
#step is the amount of features to remove in one step, cv has to do with the nr of cross validations
selector = estimator.fit(sgX_test, y_test)
#Here the model (selector) is used to make prediction for the testdata. 
#The predictions are then written to a file
        
ans2 = best[1] #answerlist[0] #selector.predict(sgtestData)
c = csv.writer(open("submissionjurretemp.csv", "wb"))
c.writerow(['ID' , 'Prediction'])
for i in range(0, len(ans2)):
    c.writerow([i+1 , ans2[i]]) 
    
    

.
.
.
.
.
.
.
.
.
.
.
.
.
(array([ 57.83149618,  41.72454266,  67.73793785,  78.31532579,
        73.32519496,  76.06227963]), 86.283614526017146)
('ansnr = ', 0)
('i en j', 5774.8201281383517, 1.1937766417144358e-08)
10811.5438885
0.700816559993
--------------------------------------
.
(array([ 57.8075189 ,  42.25359323,  67.63132284,  78.21789076,
        73.37326817,  76.01021244]), 86.796639216957828)
('ansnr = ', 1)
('i en j', 6309.5734448019302, 1.1937766417144358e-08)
10399.4395672
0.712220554627
--------------------------------------
.
(array([ 57.67980031,  42.93867841,  67.54887586,  78.15551901,
        73.25760777,  75.98016879]), 87.047468815512786)
('ansnr = ', 2)
('i en j', 6893.8453790739295, 1.1937766417144358e-08)
10068.4115493
0.721380957817
--------------------------------------
.
(array([ 58.45520016,  42.98732961,  67.22814192,  77.99517601,
        74.2327388 ,  75.8878386 ]), 83.889848767713005)
('ansnr = ', 3)
('i en j', 7532.2213975862396, 7.0170382867038367e-

In [ ]:
('i en j', 10000.0, 6.1584821106602548e-09)
9463.87010038
0.738110186518

10000.0, 7.8475997035146226e-09)
9490.19078445
0.737381824973

('i en j', 31622.776601683792, 7.8475997035146226e-09)
6794.81625451
0.811969823902

('i en j', 31622.776601683792, 4.1246263829013476e-09)
6445.69278775
0.821630975062

('i en j', 40522.133937974686, 4.1246263829013476e-09)
6334.84197833
0.824698504254

In [10]:
print(parspace)

[ 316.22776602  341.45488738  368.69450645  398.10717055  429.86623471
  464.15888336  501.18723363  541.16952655  584.34141337  630.95734448]


In [ ]:
tst = np.linspace(0,4, 7)
print(tst[0], tst[1], tst[2])

In [ ]:
np.logspace(0.9 ,2,2)

In [30]:
np.log10(23000)

4.3617278360175931

In [18]:
10**2.64

436.5158322401661

In [31]:
parspace

array([  1995.26231497,   2180.02532349,   2381.8975457 ,   2602.46330953,
         2843.45365302,   3106.75990983,   3394.44855277,   3708.77741177,
         4052.2133938 ,   4427.45184349,   4837.43769676,   5285.38859308,
         5774.82012814,   6309.5734448 ,   6893.84537907,   7532.22139759,
         8229.71158513,   8991.78996466,   9824.43745837,  10734.18882701,
        11728.18395579,  12814.22388944,  14000.83205615,  15297.32116091,
        16713.8662732 ,  18261.5846827 ,  19952.62314969,  21800.2532349 ,
        23818.97545703,  26024.63309528,  28434.53653017,  31067.59909834,
        33944.48552769,  37087.77411774,  40522.13393797,  44274.51843494,
        48374.37696758,  52853.88593079,  57748.20128138,  63095.73444802])